In [37]:
from datasets import load_dataset

raw_data = load_dataset('yitingxie/rlhf-reward-datasets')
raw_data

Found cached dataset parquet (/root/.cache/huggingface/datasets/yitingxie___parquet/yitingxie--rlhf-reward-datasets-f2627438ff1fb9dd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 219.02it/s]


DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 76256
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 5103
    })
})

In [14]:
batch = raw_data['train'][:5]
examples_list = []
for i in range(len(batch['prompt'])):
    examples_list.append({
        k:batch[k][i] for k in batch
    })
        
examples_list

[{'prompt': "\n\nHuman: Is it legal to have a dog that is half wolf?\n\nAssistant: It sounds like you’re asking whether it’s legal to have a dog that is partially wolf, as opposed to all wolf.  There’s a bit of ambiguity to your question.  What’s the context of your interest in the law?\n\nHuman: I'd like to get a puppy that is half wolf and half husky.\n\nAssistant: Ah, now I understand.  We can look up some details on the breed of wolf and husky dogs and see if they’re considered “hybrids”.\n\nHuman: They are called wolf dogs. It doesn't matter if you want to call them hybrids though.\n\nAssistant: That makes it sound like you’re seeking information to get an opinion on whether your dog is legal or not.\n\nHuman: Yes, does it depend on the state?\n\n",
  'chosen': 'Assistant: Can I ask why you’re so interested in getting a dog that is a hybrid?',
  'rejected': 'Assistant: There are a lot of state laws about “hybrid” or “mutant” dogs.  Usually a dog is defined as a certain “breed” or 

In [42]:
import re
def filtering(batch):
    res = {
        'prompt':[],
        'chosen':[],
        'rejected':[]
    }
    examples_list = []
    for i in range(len(batch['prompt'])):
        examples_list.append({
            k:batch[k][i] for k in batch
        })
            
    for example in examples_list:
        q = example['prompt']
        if len(re.findall('Human:',q)) > 1:
            continue
        else:
            q = q.replace(' Human: ','').replace('Human: ','').strip()
            chosen = example['chosen'].replace(' Assistant: ','').replace('Assistant: ','').strip()
            rejected = example['rejected'].replace(' Assistant: ','').replace('Assistant: ','').strip()
        res['prompt'].append(q)
        res['chosen'].append(chosen)
        res['rejected'].append(rejected)
    return res
            
            
filtered_data = raw_data.map(filtering, batched=True)
filtered_data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 23439
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1575
    })
})

In [43]:
filtered_data['test'][:3]

{'prompt': ['Do you know why turkeys became the official food of thanksgiving?',
  'How do I float on my back in the water?',
  "I want to make a pumpkin pie for Thanksgiving, but I've never made one before.  How do I make a pumpkin pie?"],
 'chosen': ['To be honest, I don’t know anything about that. I know that I’m meant to know a lot about history and current events, but I haven’t been programmed with those particular facts, sorry.',
  'You should tuck your arms to your sides and arch your back. Then use your back muscles to raise and lower your body.',
  'There are many variations on pumpkin pie recipes, but here’s one of my favorites:\n\n4 eggs\n\n3⁄4 cup pumpkin puree\n\n1 cup sugar\n\n1⁄2 cup brown sugar\n\n2 cups all-purpose flour\n\n1⁄2 teaspoon baking powder\n\n1⁄2 teaspoon salt\n\n1 teaspoon cinnamon\n\n1 teaspoon ground cloves\n\n1 teaspoon nutmeg\n\n1 teaspoon allspice\n\n1⁄4 teaspoon ground ginger\n\n2/3 cup butter, melted\n\nPreheat oven to 400 degrees. In a medium bowl, 

In [44]:
# 过滤之后test太少了，从train里面挪一点过去
from datasets import concatenate_datasets
filtered_data['test'] = concatenate_datasets([filtered_data['test'],filtered_data['train'].select(range(20000,23439))])
filtered_data['train'] = filtered_data['train'].select(range(0,20000))
filtered_data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 5014
    })
})

In [45]:
filtered_data.push_to_hub('beyond/rlhf-reward-single-round')

Pushing split train to the Hub.
Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]
Pushing split test to the Hub.
Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  2.10it/s]


In [ ]:
from datasets import load_dataset
d = load_dataset('beyond/rlhf-reward-single-round-trans_chinese')

In [2]:
d.save_to_disk('../data/rlhf-reward-single-round-trans_chinese')